In [1]:
import os
import csv
import random

# Define game dimensions
BOARD_DIM = 13
neighbors = [
    -(BOARD_DIM + 2) + 1, -(BOARD_DIM + 2), -1, 1,
    (BOARD_DIM + 2), (BOARD_DIM + 2) - 1
]

# Hex game class
class HexGame:
    def __init__(self):
        self.board = [0] * ((BOARD_DIM + 2) * (BOARD_DIM + 2) * 2)
        self.open_positions = [0] * (BOARD_DIM * BOARD_DIM)
        self.number_of_open_positions = 0
        self.moves = [0] * (BOARD_DIM * BOARD_DIM)
        self.connected = [0] * ((BOARD_DIM + 2) * (BOARD_DIM + 2) * 2)
        self.init_board()

    def init_board(self):
        for i in range(BOARD_DIM + 2):
            for j in range(BOARD_DIM + 2):
                self.board[(i * (BOARD_DIM + 2) + j) * 2] = 0
                self.board[(i * (BOARD_DIM + 2) + j) * 2 + 1] = 0
                if 0 < i < BOARD_DIM + 1 and 0 < j < BOARD_DIM + 1:
                    self.open_positions[(i - 1) * BOARD_DIM + j - 1] = i * (BOARD_DIM + 2) + j
                if i == 0:
                    self.connected[(i * (BOARD_DIM + 2) + j) * 2] = 1
                else:
                    self.connected[(i * (BOARD_DIM + 2) + j) * 2] = 0
                if j == 0:
                    self.connected[(i * (BOARD_DIM + 2) + j) * 2 + 1] = 1
                else:
                    self.connected[(i * (BOARD_DIM + 2) + j) * 2 + 1] = 0
        self.number_of_open_positions = BOARD_DIM * BOARD_DIM

    def connect(self, player, position):
        self.connected[position * 2 + player] = 1
        if player == 0 and position // (BOARD_DIM + 2) == BOARD_DIM:
            return True
        if player == 1 and position % (BOARD_DIM + 2) == BOARD_DIM:
            return True
        for i in range(6):
            neighbor = position + neighbors[i]
            if self.board[neighbor * 2 + player] and not self.connected[neighbor * 2 + player]:
                if self.connect(player, neighbor):
                    return True
        return False

    def winner(self, player, position):
        for i in range(6):
            neighbor = position + neighbors[i]
            if self.connected[neighbor * 2 + player]:
                return self.connect(player, position)
        return False

    def place_piece_randomly(self, player):
        random_empty_position_index = random.randint(0, self.number_of_open_positions - 1)
        empty_position = self.open_positions[random_empty_position_index]
        self.board[empty_position * 2 + player] = 1
        self.moves[BOARD_DIM * BOARD_DIM - self.number_of_open_positions] = empty_position
        self.open_positions[random_empty_position_index] = self.open_positions[self.number_of_open_positions - 1]
        self.number_of_open_positions -= 1
        return empty_position

    def full_board(self):
        return self.number_of_open_positions == 0


# Main function to generate dataset
def generate_hex_game_data():
    # Ensure the dataset folder exists
    dataset_folder = "datasett"
    os.makedirs(dataset_folder, exist_ok=True)
    file_path = os.path.join(dataset_folder, "13x13.csv")

    # Prepare column headers for board cells
    cell_columns = [f"cell{i}_{j}" for i in range(BOARD_DIM) for j in range(BOARD_DIM)]
    fieldnames = cell_columns + ["starting_player", "winner"]

    with open(file_path, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for game in range(10000):  # Adjust the number of games as needed
            hg = HexGame()
            hg.init_board()
            starting_player = random.randint(0, 1)
            player = starting_player
            winner = -1

            while not hg.full_board():
                position = hg.place_piece_randomly(player)
                if hg.winner(player, position):
                    winner = player
                    break
                player = 1 - player

            # Write the final board state and game info to the CSV
            board_state = {f"cell{i}_{j}": 0 for i in range(BOARD_DIM) for j in range(BOARD_DIM)}
            for i in range(BOARD_DIM):
                for j in range(BOARD_DIM):
                    pos = ((i + 1) * (BOARD_DIM + 2) + (j + 1)) * 2
                    if hg.board[pos] == 1:
                        board_state[f"cell{i}_{j}"] = 1
                    elif hg.board[pos + 1] == 1:
                        board_state[f"cell{i}_{j}"] = -1

            board_state["starting_player"] = starting_player
            board_state["winner"] = winner
            writer.writerow(board_state)

# Call the function
generate_hex_game_data()

print("Dataset generated in the 'dataset' folder as 'hex_game_data.csv'.")


Dataset generated in the 'dataset' folder as 'hex_game_data.csv'.
